# Triplets

Generate random triplets from pre-computed embedding vectors.

In [2]:
from livia.embedding import load_csv

# Load metadata and construct data frame from gzipped sample file
# embeddings = load_csv("../data/sentence_embeddings/bel_sbert_Title_Description_ExpertTags_256d.csv")
# embeddings = load_csv("../data/graph_embeddings/bel_graph_embedding.csv")
embeddings = load_csv("../data/sentence_embeddings/mak_sbert_title_description_256d.csv")

2022-07-27 16:05:44.232711: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-27 16:05:44.232727: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [15]:
from livia.triplet import generate_triplets

NUMBER_OF_TRIPLETS = 5000

random_triplets = generate_triplets(embeddings, method='clustering', n=NUMBER_OF_TRIPLETS)

5000 triplets are generated. This may take a while ... 
0 triplets have been removed to preserve uniqueness
5000 triplets are returned


Triplets are just tuples of IDs. Now join relevant metadata.

In [13]:
import pandas as pd

# df = pd.read_csv("../data/metadata/wien_museum.csv")
# df = pd.read_csv("../data/metadata/belvedere_flattened_20220601.csv")
df = pd.read_csv("../data/metadata/mak_minimal.csv")
df = df.astype(str)

df

,priref,url,title,reproduction,rights
0,10,https://sammlung.mak.at/sammlung_online?id=col...,Bull Spike,https://sammlung.mak.at/img/1200x1200/publikat...,bewilligt
1,100,https://sammlung.mak.at/sammlung_online?id=col...,Zeitansage,https://sammlung.mak.at/img/1200x1200/publikat...,bewilligt
2,1000,https://sammlung.mak.at/sammlung_online?id=col...,Green Medium,https://sammlung.mak.at/img/1200x1200/null,nan
3,10000,https://sammlung.mak.at/sammlung_online?id=col...,nan,https://sammlung.mak.at/img/1200x1200/null,bewilligt
4,10001,https://sammlung.mak.at/sammlung_online?id=col...,nan,https://sammlung.mak.at/img/1200x1200/publikat...,bewilligt
...,...,...,...,...,...
259593,99960,https://sammlung.mak.at/sammlung_online?id=col...,Doppelnischen-/ Medaillon-Teppich,https://sammlung.mak.at/img/1200x1200/publikat...,bewilligt
259594,9997,https://sammlung.mak.at/sammlung_online?id=col...,Dessertteller,https://sammlung.mak.at/img/1200x1200/null,bewilligt
259595,9998,https://sammlung.mak.at/sammlung_online?id=col...,Dessertteller,https://sammlung.mak.at/img/1200x1200/null,bewilligt
259596,9999,https://sammlung.mak.at/sammlung_online?id=col...,nan,https://sammlung.mak.at/img/1200x1200/null,bewilligt


In [16]:
import json

triplets_with_metadata = []

def meta(row):
  """
  # Wien Museum config
  return {
    'id': row['id'].values[0],
    'title': row['title'].values[0],
    'url': row['url'].values[0],
    'image': row['multimedia_preview'].values[0]
  }
  """

  """
  # Belvedere config
  return {
    'id': row['Identifier'].values[0],
    'title': row['Title'].values[0],
    'url': row['IsShownAt'].values[0],
    'image': row['IsShownBy'].values[0].replace('?download=', '')
  }
  """


  image = 'https://sammlung.mak.at/img/1200x1200/null'
  if (str(row['rights'].values[0]) == 'bewilligt'):
    image = row['reproduction'].values[0]


  # MAK config
  return {
    'id': row['priref'].values[0],
    'title': row['title'].values[0],
    'url': row['url'].values[0],
    'image': image
  }
  
for triplet in random_triplets:
  a, b, c = triplet

  # anchor = df.loc[df['id'] == a]
  # similar = df.loc[df['id'] == b]
  # dissimilar = df.loc[df['id'] == c]

  # anchor = df.loc[df['Identifier'] == a]
  # similar = df.loc[df['Identifier'] == b]
  # dissimilar = df.loc[df['Identifier'] == c]

  anchor = df.loc[df['priref'] == a]
  similar = df.loc[df['priref'] == b]
  dissimilar = df.loc[df['priref'] == c]

  triplets_with_metadata.append({
    'anchor': meta(anchor),
    'similar': meta(similar),
    'dissimilar': meta(dissimilar)
  })

with open('triplets.json', 'w') as outfile:
  json.dump(triplets_with_metadata, outfile)